In [1]:
import torch
import torch.nn  as  nn  
import torch.optim as  optim
from   torchvision import datasets,  transforms
import seaborn as  sns   
import matplotlib.pyplot as  plt   # Download the MNIST dataset


In [ ]:
transform = transforms.ToTensor() 
train_dataset = datasets.MNIST(root   ='./data',  train=True   ,  download=True   , transform=transform) 
test_dataset = datasets.MNIST(root   ='./data',  train=False,  download=True   , transform=transform) 
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64 ,  shuffle=True   ) 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

1.Regularization:
    a.Add L1 regularization to the 2nd layer (the layer after the input layer)
    b.Add L2 regularization instead on the 2nd layer. 
    c.What do you observe? (Hint: The lambda value used has a big impact on performance.)
    d.What is the purpose of adding regularization?

2.Dropout:
    a.Add a dropout layer between the first and second layer. What do you observe?
    b.What is the purpose of adding dropout?

3.Layers:
    a.Experiment with different amount of layers. What do you observe?
    b.Experiment with different amount of neurons in each layer. What do you observe?

4. Momentum:
    a. Try to add momentum to the SGD optimizer. 
    b. Test different values of momentum. What value do you get the highest accuracy?
    c. What happens if momentum is too high?

In [3]:
# CHECK SHAPE OF DATA INPUTS
for X, y in test_loader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Using mps device


In [ ]:

# Define model
class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.flatten = nn.Flatten();
        self.fc1 = nn.Linear(in_features=28*28, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=250)
        self.fc3 = nn.Linear(in_features=250, out_features=125)
        self.fc4 = nn.Linear(in_features=125, out_features=10)

        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        return x
model = MyNetwork().to(device)
print(model)
